<a href="https://colab.research.google.com/github/assayonare/CNN-for-assessing-the-state-of-capillaries/blob/main/capillaroscopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import os
import csv

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from tensorflow.keras.models import load_model

def merge_overlapping_boxes(boxes, overlapThresh=0.3):
    """
    Объединяет перекрывающиеся боксы.
    Каждый бокс задается в формате (x, y, w, h).
    """
    if len(boxes) == 0:
        return []

    # Преобразуем боксы в формат (x1, y1, x2, y2)
    boxes_arr = np.array([[x, y, x+w, y+h] for (x, y, w, h) in boxes])
    merged = True
    while merged:
        merged = False
        new_boxes = []
        used = np.zeros(len(boxes_arr), dtype=bool)

        for i in range(len(boxes_arr)):
            if used[i]:
                continue
            # Текущий бокс
            current_box = boxes_arr[i].copy()
            for j in range(i+1, len(boxes_arr)):
                if used[j]:
                    continue
                box = boxes_arr[j]
                # Вычисляем пересечение
                xx1 = max(current_box[0], box[0])
                yy1 = max(current_box[1], box[1])
                xx2 = min(current_box[2], box[2])
                yy2 = min(current_box[3], box[3])
                w = max(0, xx2 - xx1)
                h = max(0, yy2 - yy1)
                inter = w * h
                area_current = (current_box[2]-current_box[0]) * (current_box[3]-current_box[1])
                area_box = (box[2]-box[0]) * (box[3]-box[1])
                iou = inter / float(area_current + area_box - inter)
                if iou > overlapThresh:
                    # Объединяем боксы
                    current_box[0] = min(current_box[0], box[0])
                    current_box[1] = min(current_box[1], box[1])
                    current_box[2] = max(current_box[2], box[2])
                    current_box[3] = max(current_box[3], box[3])
                    used[j] = True
                    merged = True
            new_boxes.append(current_box)
            used[i] = True
        boxes_arr = np.array(new_boxes)
    # Возвращаем в формате (x, y, w, h)
    return [(int(x1), int(y1), int(x2-x1), int(y2-y1)) for x1, y1, x2, y2 in boxes_arr]


# Задаём входной размер, который ожидает модель (например, 64x64)
input_size = (64, 64)

# 2. Загружаем изображение капилляров в оттенках серого
image = cv2.imread("/content/drive/MyDrive/inp/image_capillar (13).png", cv2.IMREAD_GRAYSCALE)
if image is None:
    raise Exception("Не удалось загрузить изображение!")

# 3. Улучшаем изображение: фильтрация шума, обнаружение границ и поиск контуров
blurred = cv2.GaussianBlur(image, (3, 3), 0)
edges = cv2.Canny(blurred, 50, 150)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 20]

# 4. Определяем bounding boxes (прямоугольные рамки) с отступом
padding = 30  # увеличение рамки
boxes = []
for cnt in filtered_contours:
    x, y, w, h = cv2.boundingRect(cnt)
    x_new = max(0, x - padding)
    y_new = max(0, y - padding)
    w_new = min(image.shape[1] - x_new, w + 2 * padding)
    h_new = min(image.shape[0] - y_new, h + 2 * padding)
    boxes.append((x_new, y_new, w_new, h_new))

# 5. Кластеризация bounding boxes по центрам (DBSCAN)
box_centers = [(x + w // 2, y + h // 2) for x, y, w, h in boxes]
X = np.array(box_centers)
dbscan = DBSCAN(eps=35, min_samples=1).fit(X)
clustered_boxes = {}
for label, box in zip(dbscan.labels_, boxes):
    if label not in clustered_boxes:
        clustered_boxes[label] = []
    clustered_boxes[label].append(box)

# 6. Создаем объединенные bounding boxes для каждого кластера
final_boxes = []
for cluster in clustered_boxes.values():
    x_min = min(x for x, y, w, h in cluster)
    y_min = min(y for x, y, w, h in cluster)
    x_max = max(x + w for x, y, w, h in cluster)
    y_max = max(y + h for x, y, w, h in cluster)
    final_boxes.append((x_min, y_min, x_max - x_min, y_max - y_min))

# 7. Применяем дополнительное объединение, чтобы боксы не перекрывались
final_boxes = merge_overlapping_boxes(final_boxes, overlapThresh=0.3)

# 8. Готовим изображение для визуализации (конвертируем в BGR, чтобы можно было наложить цветное выделение)
image_with_overlay = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# 9. Для каждого объединённого сегмента:
#    - Извлекаем ROI, масштабируем до input_size, нормализуем и получаем предсказание модели.
#    - Если предсказание указывает на класс 0 (дефектный), выделяем область красным полупрозрачным наложением.
for (x, y, w, h) in final_boxes:
    roi = image[y:y+h, x:x+w]
    # Если модель ожидает 3 канала, преобразуем ROI в цветное изображение
    roi_color = cv2.cvtColor(roi, cv2.COLOR_GRAY2BGR)
    roi_resized = cv2.resize(roi_color, input_size)
    roi_normalized = roi_resized.astype('float32') / 255.0
    roi_input = np.expand_dims(roi_normalized, axis=0)

    # Получаем предсказание модели
    prediction = model.predict(roi_input)

    # Если значение меньше 0.5, считаем сегмент дефектным (класс 0)
    if prediction[0][0] < 0.5:
        overlay = image_with_overlay.copy()
        cv2.rectangle(overlay, (x, y), (x+w, y+h), (0, 0, 255), -1)  # заливка красным
        alpha = 0.4  # коэффициент прозрачности
        image_with_overlay = cv2.addWeighted(overlay, alpha, image_with_overlay, 1 - alpha, 0)

# 10. Отображаем результат через matplotlib
plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(image_with_overlay, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Сегменты класса 0 (без перекрывающихся боксов)")
plt.show()

In [ ]:
image = cv2.imread("/content/drive/MyDrive/inp/image_capillar (19).png", cv2.IMREAD_GRAYSCALE)

# 2. Улучшение контраста (CLAHE)
# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# image = clahe.apply(image)

# 3. Фильтрация шума
blurred = cv2.GaussianBlur(image, (3,3), 0)

# 4. Обнаружение границ (Canny)
edges = cv2.Canny(blurred, 50, 150)

# 5. Поиск контуров
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 6. Фильтрация контуров (удаляем слишком маленькие объекты)
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 20]

# 7. Определяем bounding boxes (прямоугольные рамки)
padding = 30  # Увеличение рамки
boxes = []

for cnt in filtered_contours:
    x, y, w, h = cv2.boundingRect(cnt)
    x_new = max(0, x - padding)
    y_new = max(0, y - padding)
    w_new = min(image.shape[1] - x_new, w + 2 * padding)
    h_new = min(image.shape[0] - y_new, h + 2 * padding)
    boxes.append((x_new, y_new, w_new, h_new))

# 8. Преобразуем bounding boxes в формат (x, y) центра + размеры
box_centers = [(x + w // 2, y + h // 2) for x, y, w, h in boxes]
box_sizes = [(w, h) for _, _, w, h in boxes]

# 9. Применяем DBSCAN для кластеризации близких капилляров
X = np.array(boxes)  # Массив координат центров рамок
dbscan = DBSCAN(eps=35, min_samples=1).fit(X)  # eps — радиус объединения

# 10. Группировка рамок по кластерам
clustered_boxes = {}
for label, ((x, y), (w, h)) in zip(dbscan.labels_, zip(box_centers, box_sizes)):
    if label not in clustered_boxes:
        clustered_boxes[label] = []
    clustered_boxes[label].append((x, y, w, h))

# 11. Создаем объединенные bounding boxes для каждого кластера
final_boxes = []
for cluster in clustered_boxes.values():
    x_min = min(x - w // 2 for x, y, w, h in cluster)
    y_min = min(y - h // 2 for x, y, w, h in cluster)
    x_max = max(x + w // 2 for x, y, w, h in cluster)
    y_max = max(y + h // 2 for x, y, w, h in cluster)
    final_boxes.append((x_min, y_min, x_max - x_min, y_max - y_min))

# 12. Визуализация результатов
image_with_boxes = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# 13. Вырезаем и сохраняем сегменты
segments = []
for (x, y, w, h) in final_boxes:
    segment = image[y:y+h, x:x+w]
    segments.append(segment)
    cv2.rectangle(image_with_boxes, (x, y), (x+w, y+h), (0, 255, 0), 2)

# 14. Вывод изображения с итоговыми рамками
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
plt.title("Капилляры после кластеризации DBSCAN")
plt.axis("off")
plt.show()

# 15. Вывод каждого сегмента отдельно
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i in range(5):
    axes[i].imshow(segments[i], cmap="gray")
    axes[i].axis("off")
    axes[i].set_title(f"Капилляр {i+1}")

plt.show()

In [ ]:
def save_segment(segment, segment_id, output_dir="/content/drive/MyDrive/inp/segments19"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    segment_filename = f"{output_dir}/segment_{4122+segment_id}.png"
    cv2.imwrite(segment_filename, segment)  # Сохранение изображения

segment_id = 1  # Идентификатор сегмента (можно увеличить для каждого сегмента)
for segment in segments:
    segment = cv2.resize(segment, (64, 64))
    save_segment(segment, segment_id)
    segment_id += 1

In [ ]:
def write_to_csv(segment_id, label, csv_filename="/content/drive/MyDrive/inp/labels19.csv"):
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f"segment_{4122+segment_id}.png", label])

# Пример использования
for segment_id in range(1, segment_id+1):
    label = "1"  # или "unhealthy"
    write_to_csv(segment_id, label)


In [ ]:
import pandas as pd

# Загрузка CSV-файла
df = pd.read_csv('/content/drive/MyDrive/capillaroscopy/csv/labels.csv')

# Подсчёт количества элементов в каждом классе
class_counts = df['class'].value_counts()

# Вывод результатов
print("Количество элементов каждого класса:")
print(class_counts)

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/capillaroscopy/csv/labels.csv")
df["filepath"] = df["filename"].apply(lambda x: os.path.join("/content/drive/MyDrive/capillaroscopy/data/train", x))
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df["class"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["class"], random_state=42)


X_train = np.array(train_df["filepath"])
y_train = np.array(train_df["class"])

# Oversampling редких классов
ros = RandomOverSampler(sampling_strategy="auto")  # Авто-уравнивание
X_resampled, y_resampled = ros.fit_resample(X_train.reshape(-1, 1), y_train)

# Обновляем датафрейм
train_df_resampled = pd.DataFrame({"filepath": X_resampled.flatten(), "class": y_resampled})

print(f"Train: {len(train_df_resampled)}, Validation: {len(val_df)}, Test: {len(test_df)}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)  # Без аугментации
train_df_resampled["class"] = train_df_resampled["class"].astype(str)
val_df["class"] = val_df["class"].astype(str)
test_df["class"] = test_df["class"].astype(str)
# Генераторы данных
train_generator = train_datagen.flow_from_dataframe(train_df_resampled, x_col="filepath", y_col="class", target_size=(64, 64),
                                                    batch_size=32, class_mode="sparse")
val_generator = val_test_datagen.flow_from_dataframe(val_df, x_col="filepath", y_col="class", target_size=(64, 64),
                                                     batch_size=32, class_mode="sparse")
test_generator = val_test_datagen.flow_from_dataframe(test_df, x_col="filepath", y_col="class", target_size=(64, 64),
                                                      batch_size=32, class_mode="sparse", shuffle=False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
# Загружаем EfficientNetB0 без последнего слоя
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(64, 64, 3))
# base_model.trainable = False  # Замораживаем предобученные веса
for layer in base_model.layers[:-10]:  # Можно оставить последние 10 обучаемыми
    layer.trainable = False
# Добавляем свои слои
# x = GlobalAveragePooling2D()(base_model.output)
# x = Dense(128, activation="relu")(x)
# x = Dense(3, activation="softmax")(x)  # 3 класса

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Уменьшает переобучение
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(3, activation="softmax")(x)  # 3 класса


# Финальная модель
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

#class_weights = compute_class_weight("balanced", classes=np.unique(train_df["class"]), y=train_df["class"])

# Преобразуем в словарь
#class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
history = model.fit(train_generator, validation_data=val_generator, epochs=20, steps_per_epoch=len(train_generator),
                    validation_steps=len(val_generator))
# Обучение модели с учетом весов классов
#history = model.fit(train_generator, validation_data=val_generator, epochs=20,
#                    steps_per_epoch=len(train_generator), validation_steps=len(val_generator),
#                   class_weight=class_weight_dict)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

# График точности
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, "b-", label="Точность на обучении")
plt.plot(epochs, val_acc, "r-", label="Точность на валидации")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.title("График точности")

# График потерь
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, "b-", label="Потери на обучении")
plt.plot(epochs, val_loss, "r-", label="Потери на валидации")
plt.xlabel("Эпохи")
plt.ylabel("Потери")
plt.legend()
plt.title("График потерь")

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Точность на тесте: {test_acc * 100:.2f}%")

In [ ]:
model.save("/content/drive/MyDrive/capillaroscopy/capillaries_classifier1.keras")

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/capillaroscopy/csv/labels.csv")
test_class_0_df = test_df[test_df["class"] == 1].copy()

# Добавляем полный путь к изображениям
test_class_0_df["filepath"] = test_class_0_df["filename"].apply(lambda x: os.path.join("/content/drive/MyDrive/capillaroscopy/data/train/", x))

print(f"Найдено {len(test_class_0_df)} изображений класса 0")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Генератор только для класса 0
test_class_0_generator = test_datagen.flow_from_dataframe(
    test_class_0_df, x_col="filepath", y_col=None, target_size=(64, 64),
    batch_size=32, class_mode=None, shuffle=False
)

In [ ]:
import numpy as np

In [ ]:
predictions = model.predict(test_class_0_generator)

# Берем класс с максимальной вероятностью
predicted_classes = np.argmax(predictions, axis=1)

# Выводим результаты
for filename, pred in zip(test_class_0_df["filename"], predicted_classes):
    print(f"{filename}: предсказанный класс {pred}")

Создадим кастомную CNN с 0:


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
model = Sequential([
    # 1-й сверточный блок
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # 2-й сверточный блок
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # 3-й сверточный блок
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # 4-й сверточный блок (если данных много)
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),

    # Полносвязные слои
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 3 класса
])

    # Flatten(),

#     # Полносвязные слои
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.4),
#     Dense(64, activation='relu'),
#     Dropout(0.3),
#     Dense(3, activation='softmax')  # 3 класса
# ])

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

# График точности
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, "b-", label="Точность на обучении")
plt.plot(epochs, val_acc, "r-", label="Точность на валидации")
plt.xlabel("Эпохи")
plt.ylabel("Точность")
plt.legend()
plt.title("График точности")

# График потерь
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, "b-", label="Потери на обучении")
plt.plot(epochs, val_loss, "r-", label="Потери на валидации")
plt.xlabel("Эпохи")
plt.ylabel("Потери")
plt.legend()
plt.title("График потерь")

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Точность на тесте: {test_acc * 100:.2f}%")

In [ ]:

predictions = model.predict(test_generator)


predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

filenames = test_generator.filenames

df = pd.DataFrame({
    "Filename": filenames,
    "True Class": true_classes,
    "Predicted Class": predicted_classes
})

# Сохраняем предсказания в CSV
df.to_csv("/content/drive/MyDrive/capillaroscopy/capillaries_classifier2.csv", index=False)
print("Предсказания сохранены в predictions_with_filenames.csv")

# Выводим первые 10 строк
print(df.head(10))

In [ ]:
model.save("/content/drive/MyDrive/capillaroscopy/capillaries_classifier4.keras")

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/capillaroscopy/csv/labels.csv")
test_class_0_df = test_df[test_df["class"] == 1].copy()

# Добавляем полный путь к изображениям
test_class_0_df["filepath"] = test_class_0_df["filename"].apply(lambda x: os.path.join("/content/drive/MyDrive/capillaroscopy/data/train/", x))

print(f"Найдено {len(test_class_0_df)} изображений класса 0")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Генератор только для класса 0
test_class_0_generator = test_datagen.flow_from_dataframe(
    test_class_0_df, x_col="filepath", y_col=None, target_size=(64, 64),
    batch_size=32, class_mode=None, shuffle=False
)

In [ ]:
predictions = model.predict(test_class_0_generator)

# Берем класс с максимальной вероятностью
predicted_classes = np.argmax(predictions, axis=1)

# Выводим результаты
for filename, pred in zip(test_class_0_df["filename"], predicted_classes):
    print(f"{filename}: предсказанный класс {pred}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

Делим входное изображение на части 64Х64

In [ ]:
image_path = "/content/drive/MyDrive/inp/image_capillar (15).png"  # Укажи свой путь
image = cv2.imread(image_path)

# Проверяем загрузку
if image is None:
    raise ValueError("Ошибка загрузки изображения!")

# Размеры изображения
h, w, _ = image.shape
grid_size = 15  # 15x15 = 225 частей

# Вычисляем размеры каждого блока
tile_h = h // grid_size
tile_w = w // grid_size

# Создаем папку для сохранения
output_folder = "output_tiles"
os.makedirs(output_folder, exist_ok=True)

# Разбиваем изображение и сохраняем фрагменты
tile_counter = 0
tiles = []  # Список для первых 5 фрагментов

for i in range(grid_size):
    for j in range(grid_size):
        # Вычисляем координаты текущего фрагмента
        x_start = j * tile_w
        y_start = i * tile_h
        x_end = x_start + tile_w
        y_end = y_start + tile_h

        # Вырезаем фрагмент
        tile = image[y_start:y_end, x_start:x_end]

        # Сохраняем фрагмент
        tile_filename = os.path.join(output_folder, f"tile_{tile_counter:03d}.png")
        cv2.imwrite(tile_filename, tile)

        # Добавляем первые 5 фрагментов в список
        if tile_counter < 15:
            tiles.append(cv2.cvtColor(tile, cv2.COLOR_BGR2RGB))  # Конвертируем BGR → RGB для отображения

        tile_counter += 1

fig, axes = plt.subplots(1, 15, figsize=(15, 15))
for idx, ax in enumerate(axes):
    ax.imshow(tiles[idx])
    ax.axis("off")
    ax.set_title(f"Фрагмент {idx+1}")

plt.show()

In [ ]:
from tensorflow.keras.models import load_model

# Загружаем обученную модель
model = load_model("/content/drive/MyDrive/capillaroscopy/capillaries_classifier2_func.keras")
model.summary()

In [ ]:
# Папка с разрезанными фрагментами
tile_folder = "output_tiles"
tile_size = (64, 64)  # Размер изображений, который использовала модель

# Загружаем все изображения из папки
image_files = sorted(os.listdir(tile_folder))  # Сортируем по имени
tiles = []

for filename in image_files:
    img_path = os.path.join(tile_folder, filename)
    img = cv2.imread(img_path)

    if img is None:
        continue  # Пропускаем, если файл не загрузился

    # Изменяем размер под входные данные модели
    img = cv2.resize(img, tile_size)

    # Нормализация значений пикселей
    img = img / 255.0

    # Добавляем в список
    tiles.append(img)

# Преобразуем в numpy-массив для подачи в модель
tiles = np.array(tiles)

In [ ]:
# Делаем предсказания
predictions = model.predict(tiles)

# Получаем предсказанный класс для каждого фрагмента
predicted_classes = np.argmax(predictions, axis=1)

# Выводим первые 10 предсказаний
for i in range(10):
    print(f"📌 Файл: {image_files[i]} → Класс: {predicted_classes[i]}")

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "Filename": image_files,
    "Predicted Class": predicted_classes
})

df.to_csv("/content/drive/MyDrive/capillaroscopy/tile_predictions.csv", index=False)
print("📁 Предсказания сохранены в tile_predictions.csv")

In [ ]:
original_image = cv2.imread(image_path)

if original_image is None:
    raise ValueError("Ошибка загрузки изображения!")

h, w, _ = original_image.shape  # Размер изображения

# 🔹 Папка с разрезанными частями
tile_folder = "output_tiles"
tile_size = (64, 64)  # Размер, на который натренирована модель
grid_size = 15 # Разбиение 15x15 = 225 частей

# Загружаем все изображения
image_files = sorted(os.listdir(tile_folder))
tiles = []

for filename in image_files:
    img_path = os.path.join(tile_folder, filename)
    img = cv2.imread(img_path)

    if img is None:
        continue

    # Изменяем размер под вход модели
    img = cv2.resize(img, tile_size)
    img = img / 255.0  # Нормализация
    tiles.append(img)

# Преобразуем в numpy-массив
tiles = np.array(tiles)

# 🔹 Прогоняем через модель
predictions = model.predict(tiles)
predicted_classes = np.argmax(predictions, axis=1)  # Определяем классы

# 🔹 Восстанавливаем разбиение по изображению
tile_h = h // grid_size
tile_w = w // grid_size

# 🔥 Создаем маску для выделения областей класса 0
mask = np.zeros_like(original_image, dtype=np.uint8)

tile_counter = 0
for i in range(grid_size):
    for j in range(grid_size):
        x_start, y_start = j * tile_w, i * tile_h
        x_end, y_end = x_start + tile_w, y_start + tile_h

        if predicted_classes[tile_counter] == 0:  # Если класс 0, рисуем красную маску
            cv2.rectangle(mask, (x_start, y_start), (x_end, y_end), (0, 0, 255), -1)  # Красный
        elif predicted_classes[tile_counter] == 2:
             cv2.rectangle(mask, (x_start, y_start), (x_end, y_end), (0, 255, 255), -1)

        tile_counter += 1

# 🔹 Добавляем маску к изображению
alpha = 0.4  # Прозрачность
highlighted_image = cv2.addWeighted(original_image, 1, mask, alpha, 0)

# 📌 Вывод результата
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(highlighted_image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Выделенные области класса 0")
plt.show()

In [ ]:
import pandas as pd

# Читаем исходный файл CSV
df = pd.read_csv("/content/drive/MyDrive/capillaroscopy/csv/labels.csv")

# Заменяем все значения 2 в столбце "class" на 0
df.loc[df["class"] == 2, "class"] = 1

# Сохраняем изменения в исходный CSV (можно указать новый файл)
df.to_csv("/content/drive/MyDrive/capillaroscopy/csv/labels_three.csv", index=False)